In [1]:
import pandas as pd
import numpy as np
import re
import math

from urllib.parse import urlparse
from matplotlib import pyplot as plt
import seaborn as sns

from tensorflow.keras import layers
from tensorflow import keras

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, log_loss, mean_squared_error, f1_score

In [2]:
filename = "/Users/yaeerk/Documents/NAYA/FinalProject/malicious_phish.csv"

In [3]:
#attribute contains https characters
def httpSecure(url):
    htp = urlparse(url).scheme
    match = str(htp)
    if match=='https':
        return 1
    else:
        return 0

In [4]:
def abnormal_url(url):
    hostname = urlparse(url).hostname
    hostname = str(hostname)
    match = re.search(hostname, url)
    if match:
        return 1
    else:
        return 0

In [5]:
#attribute num_digit numeric
def digit_count(url):
    digits = 0
    for i in url:
        if i.isnumeric():
            digits = digits + 1
    return digits

In [6]:
#attribute num_letter numeric
def letter_count(url):
    letters = 0
    for i in url:
        if i.isalpha():
            letters = letters + 1
    return letters

In [7]:
df = pd.read_csv(filename)
df.head()

,url,type
0,br-icloud.com.br,phishing
1,mp3raid.com/music/krizz_kaliko.html,benign
2,bopsecrets.org/rexroth/cr/1.htm,benign
3,http://www.garage-pirenne.be/index.php?option=...,defacement
4,http://adventure-nicaragua.net/index.php?optio...,defacement


In [8]:
#extract features
df['https']        = df['url'].apply(lambda i: httpSecure(i))
df['abnormal_url'] = df['url'].apply(lambda i: abnormal_url(i))
df['digits']       = df['url'].apply(lambda i: digit_count(i))
df['letters']      = df['url'].apply(lambda i: letter_count(i))

df['url_length']   = [len(x) for x in df.url] #attribute url_length numeric
df['num_tokens']   = [len(re.findall(r'\w+', x)) for x in df.url] #attribute num_tokens numeric
df['label']        = [1 if i == "benign" else 0 for i in df.type] # Converting label to int

#attribute num_symbol numeric
symbol = ['&','/','\\',';','_','@','?','-','=','.','#','%','+','$','!','*',',','//']
for i in symbol:
    name = 'num_of_' + i 
    df[name] = [x.count(i) for x in df.url]
    
df.label.value_counts()

1    428103
0    223088
Name: label, dtype: int64

In [9]:
X = df.drop(['url','label','type'],axis = 1)
y = df['label']

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 2)
X_train.head()

,https,abnormal_url,digits,letters,url_length,num_tokens,num_of_&,num_of_/,num_of_\,num_of_;,...,num_of_=,num_of_.,num_of_#,num_of_%,num_of_+,num_of_$,num_of_!,num_of_*,"num_of_,",num_of_//
510482,0,0,0,26,31,5,0,3,0,0,...,0,1,0,0,0,0,0,0,0,0
194358,0,1,6,65,86,13,0,6,0,0,...,0,1,0,0,0,0,0,0,0,1
611258,0,0,6,76,94,12,1,3,0,1,...,2,3,0,0,0,0,0,0,0,0
417382,0,0,0,34,39,6,0,4,0,0,...,0,1,0,0,0,0,0,0,0,0
225565,0,0,0,69,80,12,0,2,0,0,...,0,1,0,0,0,0,0,0,0,0


In [11]:
y_train.value_counts()

1    342538
0    178414
Name: label, dtype: int64

In [12]:
model = keras.Sequential()
model.add(keras.layers.Dense(32, activation = 'relu', input_shape = (24,)))
model.add(keras.layers.Dense(16, activation ='relu'))
model.add(keras.layers.Dense(8, activation ='relu')) 
model.add(keras.layers.Dense(1, activation ='sigmoid')) 

model.compile(optimizer = keras.optimizers.Adam(1e-3) ,loss = 'binary_crossentropy', metrics = ['acc'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 32)                800       
_________________________________________________________________
dense_1 (Dense)              (None, 16)                528       
_________________________________________________________________
dense_2 (Dense)              (None, 8)                 136       
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 9         
Total params: 1,473
Trainable params: 1,473
Non-trainable params: 0
_________________________________________________________________


In [13]:
history = model.fit(X_train, y_train, batch_size = 256, epochs = 200, validation_data = (X_test, y_test))

Epoch 1/200
2035/2035 [==============================] - 3s 1ms/step - loss: 0.3361 - acc: 0.8560 - val_loss: 0.2407 - val_acc: 0.9102
Epoch 2/200
2035/2035 [==============================] - 2s 1ms/step - loss: 0.2121 - acc: 0.9147 - val_loss: 0.1941 - val_acc: 0.9237
Epoch 3/200
2035/2035 [==============================] - 2s 1ms/step - loss: 0.1932 - acc: 0.9219 - val_loss: 0.2015 - val_acc: 0.9177
Epoch 4/200
2035/2035 [==============================] - 2s 1ms/step - loss: 0.1861 - acc: 0.9246 - val_loss: 0.1915 - val_acc: 0.9219
Epoch 5/200
2035/2035 [==============================] - 2s 1ms/step - loss: 0.1815 - acc: 0.9262 - val_loss: 0.1768 - val_acc: 0.9272
Epoch 6/200
2035/2035 [==============================] - 2s 1000us/step - loss: 0.1795 - acc: 0.9270 - val_loss: 0.1755 - val_acc: 0.9271
Epoch 7/200
2035/2035 [==============================] - 2s 996us/step - loss: 0.1778 - acc: 0.9276 - val_loss: 0.1711 - val_acc: 0.9334
Epoch 8/200
2035/2035 [===========================

Epoch 61/200
2035/2035 [==============================] - 2s 1ms/step - loss: 0.1459 - acc: 0.9405 - val_loss: 0.1542 - val_acc: 0.9379
Epoch 62/200
2035/2035 [==============================] - 2s 1ms/step - loss: 0.1454 - acc: 0.9406 - val_loss: 0.1440 - val_acc: 0.9413
Epoch 63/200
2035/2035 [==============================] - 2s 974us/step - loss: 0.1452 - acc: 0.9406 - val_loss: 0.1468 - val_acc: 0.9401
Epoch 64/200
2035/2035 [==============================] - 2s 974us/step - loss: 0.1449 - acc: 0.9407 - val_loss: 0.1443 - val_acc: 0.9419
Epoch 65/200
2035/2035 [==============================] - 2s 978us/step - loss: 0.1449 - acc: 0.9407 - val_loss: 0.1515 - val_acc: 0.9391
Epoch 66/200
2035/2035 [==============================] - 2s 961us/step - loss: 0.1449 - acc: 0.9410 - val_loss: 0.1447 - val_acc: 0.9413
Epoch 67/200
2035/2035 [==============================] - 2s 950us/step - loss: 0.1445 - acc: 0.9409 - val_loss: 0.1429 - val_acc: 0.9414
Epoch 68/200
2035/2035 [==============

KeyboardInterrupt: 

In [ ]:
model.save("url-nn")

In [ ]:
loss = history.history['loss']
val_loss = history.history['val_loss']
epoch = range(1, len(loss) + 1)
plt.plot(epoch, loss, 'y', label = 'Training loss')
plt.plot(epoch, val_loss, 'r', label = 'Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()


acc = history.history['acc']
val_acc = history.history['val_acc']
plt.plot(epoch, acc, 'y', label = 'Training accuracy')
plt.plot(epoch, val_acc, 'r', label = 'Validation accuracy')
plt.title('Training and validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

In [ ]:
#get actual metrics of train data
score_train = model.evaluate(X_train, y_train)

#get actual metrics of test data
score_test = model.evaluate(X_test, y_test)

In [ ]:
scores = [score_train[1],score_test[1]]
output = pd.DataFrame({"Model":['Train','Test'], "Score":scores})

In [ ]:
plt.figure(figsize=(10, 7))
plots = sns.barplot(x = 'Model', y = 'Score', data = output)
for bar in plots.patches:
    plots.annotate(format(bar.get_height() * 100, '.2f'),
                   (bar.get_x() + bar.get_width() / 2,
                    bar.get_height()), ha = 'center', va = 'center',
                   size = 15, xytext = (0, 8),
                   textcoords='offset points')

plt.ylabel("Score", size = 14)
plt.show()